In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze base model layers
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Found 2138 images belonging to 2 classes.
Found 459 images belonging to 2 classes.
Found 459 images belonging to 2 classes.
Epoch 1/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 631s 9s/step - accuracy: 0.7619 - loss: 0.4848 - val_accuracy: 0.9821 - val_loss: 0.1733
Epoch 2/15
 1/66 ━━━━━━━━━━━━━━━━━━━━ 8:06 7s/step - accuracy: 0.9688 - loss: 0.2113

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


66/66 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.9688 - loss: 0.2113 - val_accuracy: 0.9799 - val_loss: 0.1754
Epoch 3/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 623s 9s/step - accuracy: 0.9731 - loss: 0.1657 - val_accuracy: 0.9844 - val_loss: 0.1065
Epoch 4/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 1.0000 - loss: 0.1226 - val_accuracy: 0.9821 - val_loss: 0.1083
Epoch 5/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 625s 10s/step - accuracy: 0.9873 - loss: 0.0976 - val_accuracy: 0.9799 - val_loss: 0.0837
Epoch 6/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.9688 - loss: 0.0811 - val_accuracy: 0.9799 - val_loss: 0.0838
Epoch 7/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 581s 9s/step - accuracy: 0.9873 - loss: 0.0709 - val_accuracy: 0.9866 - val_loss: 0.0636
Epoch 8/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 1.0000 - loss: 0.0382 - val_accuracy: 0.9866 - val_loss: 0.0585
Epoch 9/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 583s 9s/step - accuracy: 0.9955 - loss: 0.0549 - val_accuracy: 0.9732 - val_loss: 0

In [3]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 107s 7s/step - accuracy: 0.9907 - loss: 0.0325
Test Loss: 0.042539678514003754
Test Accuracy: 0.9912853837013245
Validation Loss: 0.039346903562545776
Validation Accuracy: 0.9891067743301392
